In [58]:
## Data analysis libraries
import pandas as pd

## Web Scraping libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [25]:
## Following: https://www.geeksforgeeks.org/scrape-linkedin-using-selenium-and-beautiful-soup-in-python/

In [139]:
# Creating a webdriver instance
driver = webdriver.Chrome()

In [140]:
# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")

# waiting for the page to load
time.sleep(5)
 


### Login through the browser...

We could programmatically enter our credentials with something like the below

```python
username = driver.find_element(By.ID, "username")
```

In [142]:
# open the jobs URL
jobs_url = "https://www.linkedin.com/jobs/"

driver.get(jobs_url)        # this will open the link

### Search for IDs in the page:

```python
ids = driver.find_elements(By.XPATH, '//*[@id]')

for ii in ids:
    print('Tag:  ' + ii.tag_name)
    print('ID:  ' + ii.get_attribute('id'))
    print('Name:  ' + str(ii.get_attribute('name')))
```

In [165]:
#ids = driver.find_elements(By.XPATH, '//*[@id]')

#for ii in ids:
#    print('Tag:  ' + ii.tag_name)
#    print('ID:  ' + ii.get_attribute('id'))
#    print('Name:  ' + str(ii.get_attribute('name')))

In [159]:
# entering search term
search_term = driver.find_element(By.ID, "jobs-search-box-keyword-id-ember24")
search_term.send_keys("data_scientist") 
#elem = driver.find_element(By.ID, "jobs-search-box-keyword-id-ember24")
#elem.send_keys(Keys.RETURN)

In [158]:
elem = driver.find_element(By.ID, "jobs-search-box-keyword-id-ember24")
elem.send_keys(Keys.RETURN)

In [161]:
# jobs-search-box-location-id-ember24
#location_term = driver.find_element(By.ID, "jobs-search-box-location-id-ember24") 
#location_term.clear()
#location_term.send_keys("New York, United States")
#location_term.send_keys(Keys.RETURN)
#location_term.send_keys("New York, United States")

### Parse the search results page

In [162]:
def assemble_page_of_listings(soup):


    job_ids = []
    job_titles = []
    job_company = []
    job_location = []
    job_salaries = []
    
    for tag in soup.find_all(class_="job-card-list"):
        job_ids.append(tag.get('data-job-id'))
        (job_titles.append(soup.find('div', {'data-job-id': tag.get('data-job-id')})
                           .find('a', {'class': 'job-card-list__title'}).text.strip()))
        ## The company for the job is under the job-card-container__primary-description
        (job_company.append(soup.find('div', {'data-job-id': tag.get('data-job-id')})
                            .find('span', {'class': 'job-card-container__primary-description'}).text.strip()))
        ## The first item in the metadata is typically the job location:
        (job_location.append(soup.find('div', {'data-job-id': tag.get('data-job-id')})
                           .find('li', {'class': 'job-card-container__metadata-item'}).text.strip())) 
        ## The second item in the list, if present, will be the salary
        try:
            job_salary = (soup.find('div', {'data-job-id': tag.get('data-job-id')})
                               .find_all('li', {'class': 'job-card-container__metadata-item'})[1].text.strip())
            if "$" in job_salary: # if the second metadata point has a dollar sign
                if "-" in job_salary:  # If it's a salary range
                    # strip everything after the second "/yr"
                    job_salaries.append("".join(job_salary.split("/yr", 2)[:2]))
                if ("-" not in job_salary) and "yr" in job_salary:
                    # strip everything after the first "/yr"
                    job_salaries.append("".join(job_salary.split("/yr", 2)[:2]))
            else:
                job_salaries.append("No_Salary_Listed")
        except IndexError:
            job_salaries.append("No_Salary_Listed")

    job_listings = pd.DataFrame(list(zip(job_ids, job_titles, job_company, job_location, job_salaries)),
               columns =['Job_ID', 'Title', 'Company', 'Location', 'Salary'])
    return job_listings

In [163]:
src = driver.page_source

# Now using beautiful soup
soup = BeautifulSoup(src, "html.parser")

job_listings = assemble_page_of_listings(soup)

In [164]:
job_listings

,Job_ID,Title,Company,Location,Salary
0,3722171802,"Data Scientist, Lead",Booz Allen Hamilton,"Norfolk, VA (Remote)",$106K - $242K
1,3725737021,"Sr. Staff Data Scientist, Ecosystem",Pinterest,"California, United States (Remote)",No_Salary_Listed
2,3718232807,Senior Staff Data Scientist,Coupang,"Seattle, WA (Hybrid)",$170K - $318K
3,3725128742,Sr. Data Scientist,Fanatics,"New York, NY (On-site)",$190K - $225K
4,3713421109,Data Scientist Analyst - Early Career,Lockheed Martin,"Fort Worth, TX",No_Salary_Listed
5,3724802079,"Principal Data Scientist, The Lab",Capital One,"San Francisco, CA",No_Salary_Listed
6,3730760978,"Sr. Program Manager, Tech (Chief of Staff) - P...",Uber,"Seattle, WA",$140K - $172K


### Click into an individual Item

```python
clickable = driver.findElement(By.id("clickable"));
clickable.click()
```

## Go to each page and get the job text


In [ ]:
# open the jobs URL
jobs_url = "https://www.linkedin.com/jobs/"

driver.get(jobs_url)        # this will open the link